In [16]:
#coding=utf-8
import xlwings as xw
import re
import pandas as pd
import psycopg2
from modules.my_libs import han_ji_chu_im as ji

#==========================================================
# Excel 檔案
#==========================================================
file_path = ('to-tek-keng.xlsx')
wb = xw.Book(file_path)

# source_sheet = wb.sheets['ji-im-tui-chiau']
source_sheet = wb.sheets['工作表1']
end_row = source_sheet.range('A' + str(source_sheet.cells.last_cell.row)).end('up').row
print(f'end_row = {end_row}')

#-----------------------------------------------------
ji_khoo_sheet  = wb.sheets.add(name="字庫表")
khiam_ji_sheet  = wb.sheets.add(name="缺字表")
chu_im_sheet = wb.sheets.add(name="漢字注音表")

#==========================================================
# 資料庫
#==========================================================
conn = psycopg2.connect(database="alanjui", user="alanjui", host="127.0.0.1", port="5432")
cur = conn.cursor()

end_row = 81


In [17]:
row = 1     # index for source sheet
chu_im_index = 1
khiam_ji_index = 1

while row < end_row:
    # 自 source_sheet 取待注音漢字
    han_ji = str(source_sheet.range('A' + str(row)).value)
    han_ji.strip()
    
    #=========================================================
    # 將【整段】讀入的漢字，轉成：每個儲存格，放一個漢字
    #=========================================================
    han_ji_list = ji.convert_string_to_list(han_ji)
    
    chu_im_sheet.range('A' + str(chu_im_index)) \
                .options(transpose=True).value = han_ji_list
    row += 1

    #=========================================================
    # 將漢字加注音
    #=========================================================
    i = chu_im_index
    # 取用每一個漢字，自【字庫】資料庫查找其【注音碼】
    for search_han_ji in han_ji_list:
        # 若取出之字為標點符號，則跳過，並繼續取下一個漢字。
        # searchObj = re.search( r'[，、；。？！（）-「」【】《》“]', search_han_ji, re.M|re.I)
        searchObj = re.search( r'[，、：；。？！（）「」【】《》“]', search_han_ji, re.M|re.I)
        if searchObj:
            i += 1
            continue
        
        # SQL 查詢指令：自字庫查找某漢字之注音碼
        #sql = f"select id, han_ji, chu_im, freq, siann, un, tiau from han_ji where han_ji='{search_han_ji}'"
        sql = "SELECT id, han_ji, chu_im, freq, siann, un, tiau "\
                "FROM han_ji "\
                f"WHERE han_ji='{search_han_ji}' "
        cur.execute(sql)
        query_rows = cur.fetchall()
        
        # 判斷是否發生：找不到漢字注音碼的問題
        if not query_rows:
            print(f"Can not find 【{search_han_ji}】in Han-Ji-Khoo!!")
            khiam_ji_sheet.range('A' + str(khiam_ji_index)).value = search_han_ji
            khiam_ji_index += 1
            i += 1
            continue
 
        # 將已找到注音碼的漢字，分解出：聲母、韻母、調號
        chu_im = query_rows[0][2]
        chu_im_code = ji.split_chu_im(chu_im)
        if chu_im_code[0] == "q":
            siann_bu = ""
        else:
            siann_bu = chu_im_code[0]
        un_bu = chu_im_code[1]
        tiau_ho = chu_im_code[2]
        
        # 取得聲母之聲母碼；及取得韻母之韻母碼；以利後續轉換：十五音、方音符號、
        # 白話字、羅馬拼音、閩拼時使用。
        if siann_bu != "":
            siann_code = ji.get_siann_idx(siann_bu)
        un_code = ji.get_un_idx(un_bu)
        
        #=========================================================
        # 將已注音之漢字加入【漢字注音表】
        #=========================================================
        chu_im_sheet.range('B' + str(i)).value = chu_im
        chu_im_sheet.range('C' + str(i)).value = siann_bu
        chu_im_sheet.range('D' + str(i)).value = un_bu
        chu_im_sheet.range('E' + str(i)).value = tiau_ho
        
        i += 1

        #=========================================================
        # 將已注音之漢字加入【字庫表】
        #=========================================================

    #=========================================================
    # 調整讀取來源；寫入標的各手標
    #=========================================================
    chu_im_index += len(han_ji_list)
    row += 1

Can not find 【埏】in Han-Ji-Khoo!!
Can not find 【埴】in Han-Ji-Khoo!!
Can not find 【楗】in Han-Ji-Khoo!!
Can not find 【琭】in Han-Ji-Khoo!!
Can not find 【琭】in Han-Ji-Khoo!!


ValueError: 'oenn' is not in list

In [ ]:
#==========================================================
# 關閉資料庫
#==========================================================
conn.close()